# HackerRank Scrapper 

### 1. CONNECT THE DRIVER

In [16]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
import re
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.firefox.options import Options

# Configure Firefox options
options = Options()
#options.add_argument("--headless") 
#options.set_preference("permissions.default.image", 2)  # Prevent images from loading

service = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=service, options=options)

# Now proceed with your web navigation
driver.get('https://www.hackerrank.com/access-account/')

time.sleep(8)

try:
    ad = driver.find_element(By.CSS_SELECTOR, "a.hr_get_started_popup-close[href='#close_get_started_popup']")

    # Scroll the button into view using JavaScript
    driver.execute_script("arguments[0].scrollIntoView(true);", ad)
    ad.click()
except Exception as e:
    print(" ")

button_login = driver.find_element(By.CSS_SELECTOR, "a.hr_button[href='/login/']")

# Scroll the button into view using JavaScript
driver.execute_script("arguments[0].scrollIntoView(true);", button_login)
time.sleep(1)
button_login.click()

time.sleep(4)

# Text you want to insert into the input fields
a = "maymon1998123@gmail.com" #USERNAME
b = "MAYesha1999" #PASSWORD

# Locate the input fields and insert text
input_1 = driver.find_element(By.ID, "input-1")
input_1.clear()  # Clear the input field before inserting text
input_1.send_keys(a)

input_2 = driver.find_element(By.ID, "input-2")
input_2.clear()  # Clear the input field before inserting text
input_2.send_keys(b)

btn = driver.find_element(By.XPATH, "//div/span[contains(text(), 'Log In')]")
driver.execute_script("arguments[0].scrollIntoView(true);", btn)
# Click on the button
btn.click()
time.sleep(5)

### 2. GET ALL QUESTION LINKS FROM A PAGE
--- Manually go to a page then execute this cell

In [ ]:
#### GET ALL QUESTION LINKS FROM A PAGE/TOPIC
from selenium.webdriver.common.keys import Keys

number_of_links_to_click = 1000  # Example, replace with user input as needed

# Fetch all links at the start (assuming they are all available on the page load)
all_links = driver.find_elements(By.CSS_SELECTOR, "div.challenges-list a")

# Prepare a list to store the URLs
stored_urls = []

# Initially define a variable to track the number of processed links
processed_links = 0

# Scroll and load until you have enough links processed
while processed_links < number_of_links_to_click:
    # Try to ensure there are enough links in view to be processed
    if processed_links >= len(all_links):
        # Scroll down to trigger lazy loading
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for the page to load more items
        time.sleep(15)  # Adjust based on your page's loading time

        # Refetch all links after scrolling
        all_links = driver.find_elements(By.CSS_SELECTOR, "div.challenges-list a")

    
    # Iterate over the links that haven't been processed yet
    for link in all_links[processed_links:]:
        # Open the link in a new tab
        link.send_keys(Keys.COMMAND + Keys.RETURN)
        time.sleep(3)  # Allow time for the new tab to open

        # Switch to the new tab (it should be the last one)
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(3)  # Allow time for the page to fully load

        # Now that the page has loaded, grab the URL
        current_url = driver.current_url
        stored_urls.append(current_url)

        # Close the current tab and switch back to the original tab
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

        # Increment the processed links counter
        processed_links += 1

        # Break if we've reached the desired number of links to process
        if processed_links >= number_of_links_to_click:
            break

In [ ]:
import pandas as pd
dataframe = pd.DataFrame(stored_urls)
dataframe.to_csv('Link-REGEX.csv')

### 3. Get all Test cases for the extracted links

In [ ]:
### GET TEST CASES
dff = pd.read_csv('ALGO.csv')
miss = 1
extract = []
for m in range(len(dff)):
    try:
        x = dff.loc[m, 'Links']  
        driver.get(x)
        input_blocks = driver.find_elements(By.CSS_SELECTOR, "div.msB.challenge_sample_input_body")

        # List to store all extracted sample inputs
        sample_inputs = []

        for block in input_blocks:
            # Within each block, find all <pre> tags
            pre_elements = block.find_elements(By.TAG_NAME, "pre")
            for pre in pre_elements:
                # Get the text within each <pre> tag and add it to our list
                text = pre.text
                sample_inputs.append(text)

        output_blocks = driver.find_elements(By.CSS_SELECTOR, "div.msB.challenge_sample_output_body")

        # List to store all extracted sample inputs
        sample_outputs = []

        for block in output_blocks:
            # Within each block, find all <pre> tags
            pre_elements = block.find_elements(By.TAG_NAME, "pre")
            for pre in pre_elements:
                # Get the text within each <pre> tag and add it to our list
                text = pre.text
                sample_outputs.append(text)

        extract.append([sample_inputs[0], sample_outputs[0]]) 
        time.sleep(2)
    except Exception as e:
        extract.append([m+1, miss])
        miss+=1
        continue


In [45]:
s = pd.DataFrame(extract)
s.to_csv('Test_Cases.csv')

### 4. GET THE LEAST AND MOST OPTIMIZED CODE for each extracted link using Binary Search

In [17]:
### GET LEAST AND MOST OPTIMIZED CODE VIA Binary Search
dff = pd.read_csv('ALGO.csv')       
# Iterate over the number of links you want to click or the number of available links, whichever is smaller
store = []
for m in range(len(dff)):

    try:
        x = dff.loc[m, 'Links']  
        driver.get(x)
        #link.send_keys(Keys.COMMAND + Keys.RETURN)  # Use Keys.COMMAND on Mac

        # Wait a bit for the new tab to open
        time.sleep(2)


        #------------------------------------------------------------------------------------------------------------------------------------

        count = 0
        time.sleep(2)
        submission_btn = driver.find_element(By.XPATH, "//a/span[contains(text(), 'Submissions')]")
        driver.execute_script("arguments[0].scrollIntoView(true);", submission_btn)
        time.sleep(0.5)
        # Click on the button
        submission_btn.click()

        try:

            time.sleep(0.3) 
            link = driver.find_element(By.XPATH, "//div/span[contains(text(), 'Editorial')]")
            link.click()
            time.sleep(0.2)
            link = driver.find_element(By.XPATH, "//div/span[contains(text(), 'Yes, I want to unlock')]")
            driver.execute_script("arguments[0].scrollIntoView(true);", link)
            link.click()

        except Exception as e:
            print('')
        time.sleep(0.3) 


        LeaderBoard_btn = driver.find_element(By.XPATH, "//a/div/span[contains(text(), 'Leaderboard')]")

        # Click on the button
        LeaderBoard_btn.click()
        time.sleep(0.3) 
        try:
            link = driver.find_element(By.XPATH, "//div/span[contains(text(), 'Reveal solutions')]")
            link.click()
            time.sleep(0.5)
            link = driver.find_element(By.XPATH, "//div/span[contains(text(), 'Yes')]")
            link.click()

        except Exception as e:
            print('')
        time.sleep(0.5) 


        original_window = driver.current_window_handle
  
        time.sleep(1)
        #ROW SIZE 100

        from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

        current_url = driver.current_url

        # Step 2: Parse and modify the URL
        parsed_url = urlparse(current_url)
        query_params = parse_qs(parsed_url.query)

        # Update 'limit' parameter to 100
        query_params['limit'] = ['100']  # Use a list to ensure proper formatting
        query_params['filter'] = ['java']  # Set filter to 'java'
        query_params['filter_on'] = ['language']  # Set filter_on to 'language'

        # Encode the updated query parameters back into a query string
        updated_query_string = urlencode(query_params, doseq=True)

        # Reconstruct the URL with the updated query parameters
        updated_url = urlunparse(parsed_url._replace(query=updated_query_string))

        #driver.switch_to.window(driver.window_handles[1])
        # Step 3: Load the updated URL in the same tab
        driver.get(updated_url)



        # Regex pattern to match "Java" followed by a version number
        java_pattern = re.compile(r"Java\s+\d+")

        rows = driver.find_elements(By.CSS_SELECTOR, "div.table-row-wrapper")
        elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'table-row-column') and contains(@class, 'ellipsis') and contains(@class, 'score')]/span[@data-balloon]")

        for i in range(100):
            div = rows[i].find_elements(By.CSS_SELECTOR, "div.ellipsis")
            if java_pattern.search(div[4].text):
                button = rows[i].find_element(By.CSS_SELECTOR, "div > div > span > a")
                driver.execute_script("arguments[0].scrollIntoView(true);", button)
                time.sleep(1)
                button.click()
                break



        time.sleep(0.5) 
        new_window = None
        for handle in driver.window_handles:
            if handle != original_window:
                new_window = handle
                break

        if new_window:
            driver.switch_to.window(driver.window_handles[-1])
            # Now you are in the new tab

            # Step 4: Wait for the element to be present and copy its text
            wait = WebDriverWait(driver, 10)
            code_element = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/pre")))
            code_text = code_element.text  # This is the text you want to copy

            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        else:
            print("No new window opened.")        

        #------------------------------------------------------------------------------------------------------------------------------------
        #GET TOTAL PAGES
        pagination_list = driver.find_element(By.CSS_SELECTOR, 'ul.pagination-list')
        pagination_items = pagination_list.find_elements(By.CSS_SELECTOR, 'li.pagination-item')
        second_last_pagination_item = pagination_items[-2]  # Python's negative indexing
        href = second_last_pagination_item.find_element(By.CSS_SELECTOR, 'a.pagination-button').get_attribute('href')
        match = re.search(r'page=(\d+)', href)
        if match:
            page_number = match.group(1)
            print(page_number)  # This will print the number after "page="
        else:
            print("Page number not found")


        def binary_search_condition_check(url_template, mid, opt_score):

            # Load the page
            # Step 2: Parse and modify the URL
            parsed_url = urlparse(url_template)
            query_params = parse_qs(parsed_url.query)
            query_params['page'] = [mid]  # Use a list to ensure proper formatting
            updated_query_string = urlencode(query_params, doseq=True)
            updated_url = urlunparse(parsed_url._replace(query=updated_query_string))
            driver.get(updated_url)
            #time.sleep(2)
            # Implement the logic to check the condition on the page
            # For example, checking if a certain element exists or contains a specific value
            rows = driver.find_elements(By.CSS_SELECTOR, "div.table-row-wrapper")
            elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'table-row-column') and contains(@class, 'ellipsis') and contains(@class, 'score')]/span[@data-balloon]")
            optimum_value= optimum_score


            y = 99
            stop = False
            stop_last = False
            while not stop_last and y >= 0:  # Ensures y doesn't go below 0
                try:
                    last_div = rows[y].find_elements(By.CSS_SELECTOR, "div.ellipsis")
                    if java_pattern.search(last_div[4].text):  # Assuming you're searching for something specific in the text
                        stop_last = True  # Found the matching element, set stop to True to exit the loop
                    else:
                        y -= 1  # Decrement y to check the previous element in the next iteration
                except IndexError:
                    # This catches cases where y is out of bounds for rows or last_div doesn't have enough elements
                    y -= 1  # Decrement y and try again        

            track = 0
            for i in range(100):
                track = track + 1
                div = rows[i].find_elements(By.CSS_SELECTOR, "div.ellipsis")
                if java_pattern.search(div[4].text):
                    track = track - 1
                    if elements[i].text == optimum_value and elements[y].text == optimum_value:
                        condition_met = False
                        #track = track - 1
                        break
                    elif elements[i].text == optimum_value and elements[y].text != optimum_value:
                        #track = track - 1
                        continue
                    elif elements[i].text != optimum_value and elements[i-(1 + track)].text == optimum_value:
                        stop = True
                        button = rows[i-(1+track)].find_element(By.CSS_SELECTOR, "div > div > span > a")
                        driver.execute_script("arguments[0].scrollIntoView(true);", button)
                        time.sleep(0.3)
                        button.click()
                        condition_met = None
                        time.sleep(0.3) 
                        new_window = None

                        for handle in driver.window_handles:
                            if handle != original_window:
                                new_window = handle
                                break

                        if new_window:
                            driver.switch_to.window(driver.window_handles[-1])
                            # Now you are in the new tab

                            # Step 4: Wait for the element to be present and copy its text
                            wait = WebDriverWait(driver, 10)
                            code_element = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/pre")))
                            code_text_G = code_element.text  # This is the text you want to copy

                            # Step 5: Print the text
                            #store.append([code_text])
                            #store.append([code_text.strip().split('\n')])
                            store.append([{'BEST': code_text.strip().split('\n'), 'GROUND': code_text_G.strip().split('\n')}])
                            print(f"{m} - APPENDEEED")
                            #print(code_text)

                            # Increment the count
                            #count += 1


                            # Step 6: Close the new tab and switch back to the original window
                            driver.close()
                            driver.switch_to.window(driver.window_handles[0])
                        else:
                            print("No new window opened.")        

                        break

                    elif elements[i].text != optimum_value:
                        condition_met = True
                        break 


            # Return True if the condition to move left is met, False if moving right is needed
            return condition_met, stop

        def binary_search_pages(url_template, total_pages, opt_score):
            low = 1
            high = total_pages

            while low <= high:
                mid = (low + high) // 2
                a, b = binary_search_condition_check(url_template, mid, opt_score)
                if b==True:
                    break
                if a==True:
                    high = mid - 1  # Move left
                else:
                    low = mid + 1   # Move right

            # Optional: Return the page number where the item was found, or ind icate not found
            return None

        # Example usage:
        url_template = driver.current_url
        #EXTRACT MOST OPTIMIZED SOLUTION SCORE
        driver.get(url_template)
        rows = driver.find_elements(By.CSS_SELECTOR, "div.table-row-wrapper")
        for row in rows:

            try:
                elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'table-row-column') and contains(@class, 'ellipsis') and contains(@class, 'score')]/span[@data-balloon]")
                if elements:  # Check if the list is not empty
                    optimum_score = elements[0].text
                    #print(optimum_score)# Click the found button

                break  # Exit the loop after clicking the first matching button
            except Exception as e:
                print(f"Couldn't click the button due to: {e}")
                continue


        total_pages = int(page_number)  # Assuming you've determined the total number of pages somehow
        found_page = binary_search_pages(url_template, total_pages, optimum_score)
        print("DONE!")
        #---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

        time.sleep(1)  
   
    except Exception as e:
        store.append([{'BEST': ' ', 'GROUND': ' '}])
        if len(driver.window_handles) == 2:
            driver.close()  # Close the current window
            driver.switch_to.window(driver.window_handles[0])

        print("SKIP")
        continue



No new window opened.
5
SKIP


No new window opened.
2
SKIP


2223
183 - APPENDEEED
DONE!


1454
184 - APPENDEEED
DONE!


1649
185 - APPENDEEED
DONE!


1339
186 - APPENDEEED
DONE!


1215
187 - APPENDEEED
DONE!


1189
188 - APPENDEEED
DONE!


867
189 - APPENDEEED
DONE!

746
190 - APPENDEEED
DONE!


878
191 - APPENDEEED
DONE!

SKIP


458
193 - APPENDEEED
DONE!

529
194 - APPENDEEED
DONE!

236
195 - APPENDEEED
DONE!

310
196 - APPENDEEED
DONE!

251
197 - APPENDEEED
DONE!

387
198 - APPENDEEED
DONE!

248
199 - APPENDEEED
DONE!

207
200 - APPENDEEED
DONE!

273
201 - APPENDEEED
DONE!

883
202 - APPENDEEED
DONE!

208
203 - APPENDEEED
DONE!

575
204 - APPENDEEED
DONE!

199
205 - APPENDEEED
DONE!

237
206 - APPENDEEED
DONE!

144
207 - APPENDEEED
DONE!

189
208 - APPENDEEED
DONE!

198
209 - APPENDEEED
DONE!

477
210 - APPENDEEED
DONE!

430
211 - APPENDEEED
DONE!

No new window opened.
165
SKIP

SKIP


No new window opened.
180
SKIP

260
215 - APPENDEEED
DONE!

101
216 - APPENDEEED
DONE!

110
2

In [18]:
import pandas as pd
list_of_dicts = [item for sublist in store for item in sublist]

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(list_of_dicts)

# Save the DataFrame to a CSV file
df.to_csv('demo.csv', index=False)
df

,BEST,GROUND
0,,
1,,
2,"[import java.io.*;, import java.util.*;, impor...","[import java.io.*;, import java.math.*;, impor..."
3,"[import java.io.*;, import java.util.*;, impor...","[import java.io.*;, import java.util.*;, , cla..."
4,"[import java.io.*;, import java.util.*;, impor...","[import java.io.*;, import java.math.*;, impor..."
...,...,...
252,"[import java.io.IOException;, import java.io.O...","[import java.io.*;, import java.math.*;, impor..."
253,"[import java.io.ByteArrayInputStream;, import ...","[import java.io.*;, import java.util.Scanner;,..."
254,,
255,"[import java.util.*;, , class MinimumOperation...","[import java.util.*;, , class MinimumOperation..."


### 5. ROUGH

In [ ]:
store = []
count = 0

submission_btn = driver.find_element(By.XPATH, "//a/span[contains(text(), 'Submissions')]")

# Click on the button
submission_btn.click()

time.sleep(0.5) 


try:
    link = driver.find_element(By.XPATH, "//div/span[contains(text(), 'Editorial')]")
    link.click()
    time.sleep(0.2)
    link = driver.find_element(By.XPATH, "//div/span[contains(text(), 'Yes, I want to unlock')]")
    driver.execute_script("arguments[0].scrollIntoView(true);", link)
    link.click()

except Exception as e:
    print('')
time.sleep(0.3) 


LeaderBoard_btn = driver.find_element(By.XPATH, "//a/div/span[contains(text(), 'Leaderboard')]")

# Click on the button
LeaderBoard_btn.click()
time.sleep(0.3) 

try:
    link = driver.find_element(By.XPATH, "//div/span[contains(text(), 'Reveal solutions')]")
    link.click()
    time.sleep(0.5)
    link = driver.find_element(By.XPATH, "//div/span[contains(text(), 'Yes')]")
    link.click()

except Exception as e:
    print('')
time.sleep(0.3) 





original_window = driver.current_window_handle



#ENTER JAVA
from selenium.webdriver.common.keys import Keys

c = "java" #Filter
input_element = driver.find_element(By.CSS_SELECTOR, ".details-filter.filter-section div div div .ac-input")

# Enter text into the input field
input_element.send_keys(c)
time.sleep(1)
# Simulate pressing the Enter key
input_element.send_keys(Keys.ENTER)
input_element.clear()

#ROW SIZE 100
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse
# No need to import `webdriver` if you've already set it up

# Assuming `driver` is your Selenium WebDriver instance and it's already at the given URL

# Step 1: Extract the current URL
current_url = driver.current_url

# Step 2: Parse and modify the URL
parsed_url = urlparse(current_url)
query_params = parse_qs(parsed_url.query)

# Update 'limit' parameter to 100
query_params['limit'] = ['100']  # Use a list to ensure proper formatting

# Encode the updated query parameters back into a query string
updated_query_string = urlencode(query_params, doseq=True)

# Reconstruct the URL with the updated query parameters
updated_url = urlunparse(parsed_url._replace(query=updated_query_string))

# Step 3: Load the updated URL in the same tab
driver.get(updated_url)



# Regex pattern to match "Java" followed by a version number
java_pattern = re.compile(r"Java\s+\d+")
      
rows = driver.find_elements(By.CSS_SELECTOR, "div.table-row-wrapper")
elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'table-row-column') and contains(@class, 'ellipsis') and contains(@class, 'score')]/span[@data-balloon]")

for i in range(100):
    div = rows[i].find_elements(By.CSS_SELECTOR, "div.ellipsis")
    if java_pattern.search(div[4].text):
        button = rows[i].find_element(By.CSS_SELECTOR, "div > div > span > a")
        driver.execute_script("arguments[0].scrollIntoView(true);", button)
        time.sleep(1)
        button.click()
        break
        
        
        
time.sleep(0.5) 
new_window = None
for handle in driver.window_handles:
    if handle != original_window:
        new_window = handle
        break

if new_window:
    driver.switch_to.window(new_window)
    # Now you are in the new tab
    
    # Step 4: Wait for the element to be present and copy its text
    wait = WebDriverWait(driver, 10)
    code_element = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/pre")))
    code_text = code_element.text  # This is the text you want to copy
    
    # Step 5: Print the text
    store.append([code_text])
    #print(code_text)
    
    # Increment the count
    #count += 1
    
    
    # Step 6: Close the new tab and switch back to the original window
    driver.close()
    driver.switch_to.window(original_window)
else:
    print("No new window opened.")        
                

#### Binary Search -  Least Optimized

In [35]:
#GET TOTAL PAGES
pagination_list = driver.find_element(By.CSS_SELECTOR, 'ul.pagination-list')
pagination_items = pagination_list.find_elements(By.CSS_SELECTOR, 'li.pagination-item')
second_last_pagination_item = pagination_items[-2]  # Python's negative indexing
href = second_last_pagination_item.find_element(By.CSS_SELECTOR, 'a.pagination-button').get_attribute('href')
match = re.search(r'page=(\d+)', href)
if match:
    page_number = match.group(1)
    print(page_number)  # This will print the number after "page="
else:
    print("Page number not found")


def binary_search_condition_check(url_template, mid, opt_score):
    
    # Load the page
    # Step 2: Parse and modify the URL
    parsed_url = urlparse(url_template)
    query_params = parse_qs(parsed_url.query)
    query_params['page'] = [mid]  # Use a list to ensure proper formatting
    updated_query_string = urlencode(query_params, doseq=True)
    updated_url = urlunparse(parsed_url._replace(query=updated_query_string))
    driver.get(updated_url)
    #time.sleep(2)
    # Implement the logic to check the condition on the page
    # For example, checking if a certain element exists or contains a specific value
    rows = driver.find_elements(By.CSS_SELECTOR, "div.table-row-wrapper")
    elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'table-row-column') and contains(@class, 'ellipsis') and contains(@class, 'score')]/span[@data-balloon]")
    optimum_value= optimum_score

    y=99
    stop = False
    for i in range(100):
        last_div = rows[y].find_elements(By.CSS_SELECTOR, "div.ellipsis")
        if java_pattern.search(last_div[4].text):
            break
        else:
            y = y - 1
 
    track = 0
    for i in range(100):
        track = track + 1
        div = rows[i].find_elements(By.CSS_SELECTOR, "div.ellipsis")
        if java_pattern.search(div[4].text):
            track = track - 1
            if elements[i].text == optimum_value and elements[y].text == optimum_value:
                condition_met = False
                #track = track - 1
                break
            elif elements[i].text == optimum_value and elements[y].text != optimum_value:
                #track = track - 1
                continue
            elif elements[i].text != optimum_value and elements[i-(1 + track)].text == optimum_value:
                stop = True
                button = rows[i-(1+track)].find_element(By.CSS_SELECTOR, "div > div > span > a")
                driver.execute_script("arguments[0].scrollIntoView(true);", button)
                time.sleep(0.3)
                button.click()
                condition_met = None
                time.sleep(0.3) 
                new_window = None
                for handle in driver.window_handles:
                    if handle != original_window:
                        new_window = handle
                        break

                if new_window:
                    driver.switch_to.window(new_window)
                    # Now you are in the new tab

                    # Step 4: Wait for the element to be present and copy its text
                    wait = WebDriverWait(driver, 10)
                    code_element = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/pre")))
                    code_text = code_element.text  # This is the text you want to copy

                    # Step 5: Print the text
                    store.append([code_text])
                    print(code_text)

                    # Increment the count
                    #count += 1


                    # Step 6: Close the new tab and switch back to the original window
                    driver.close()
                    driver.switch_to.window(original_window)
                else:
                    print("No new window opened.")        

                break
                
            elif elements[i].text != optimum_value:
                condition_met = True
                break 

    
    # Return True if the condition to move left is met, False if moving right is needed
    return condition_met, stop

def binary_search_pages(url_template, total_pages, opt_score):
    low = 1
    high = total_pages
    
    while low <= high:
        mid = (low + high) // 2
        a, b = binary_search_condition_check(url_template, mid, opt_score)
        if b==True:
            break
        if a==True:
            high = mid - 1  # Move left
        else:
            low = mid + 1   # Move right
    
    # Optional: Return the page number where the item was found, or ind icate not found
    return None

# Example usage:
url_template = driver.current_url
#EXTRACT MOST OPTIMIZED SOLUTION SCORE
driver.get(url_template)
rows = driver.find_elements(By.CSS_SELECTOR, "div.table-row-wrapper")
for row in rows:
    # Finding the <div> that directly contains the text
    #text_div = row.find_elements(By.CSS_SELECTOR, "div > div > div")  # Adjust if necessary
    #for div in text_div:
            # Once matched, find the <a> element within a <span> under the same row
    try:
        elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'table-row-column') and contains(@class, 'ellipsis') and contains(@class, 'score')]/span[@data-balloon]")
        if elements:  # Check if the list is not empty
            optimum_score = elements[0].text
            #print(optimum_score)# Click the found button

        break  # Exit the loop after clicking the first matching button
    except Exception as e:
        print(f"Couldn't click the button due to: {e}")
        continue
        
        
total_pages = int(page_number)  # Assuming you've determined the total number of pages somehow
found_page = binary_search_pages(url_template, total_pages, optimum_score)
print("DONE!")

31
import java.io.*;
import java.util.*;
import java.text.*;
import java.math.*;
import java.util.regex.*;

public class Solution {
    
    public static class Node {
        int id;
        //Neighbor->distance
        HashMap<Node,Integer> Neighbor; 
        int distance;
        
        public Node(int id){
            this.id=id;
            this.Neighbor=new HashMap<Node,Integer>();
            this.distance=-1;
        }
        
        
    }
    
    //return the index with minimum distance in the arraylsit O(Q.size())
    public static int minIndex(ArrayList<Node> Q){
        int output=-1;
        int minDistance=65536000;
        for (int i=0;i<Q.size();i++){
            if (Q.get(i).distance<minDistance){
                output=i;
                minDistance=Q.get(i).distance;
            }
            
        }
        return output;
    }

    public static void main(String[] args) {
        /* Enter your code here. Read input from STDIN. Print output to STDOUT. Your 